## Arranging Raw Images before Running SDFRED2

### Packages & Directories

In [1]:
import numpy as np
import glob, os
import pandas as pd
from pathlib import Path

In [2]:
dir_Raw = str(Path("./Raw").absolute())+"/"
dir_Flat = str(Path("./Flat").absolute())+"/"
dir_Obj = str(Path("./Object").absolute())+"/"

### Reading the data frame of raw images

In [3]:
# Loading data
df = pd.read_csv("raw_info.csv")
df.head(50)

,EXP-ID,FRAMEID,DATE-OBS,DATA-TYP,OBJECT,FILTER01,EXPTIME
0,SUPE01017660,SUPA01017660,2008-08-29,DOMEFLAT,DOMEFLAT,W-C-IC,15.0
1,SUPE01017660,SUPA01017661,2008-08-29,DOMEFLAT,DOMEFLAT,W-C-IC,15.0
2,SUPE01017660,SUPA01017662,2008-08-29,DOMEFLAT,DOMEFLAT,W-C-IC,15.0
3,SUPE01017660,SUPA01017663,2008-08-29,DOMEFLAT,DOMEFLAT,W-C-IC,15.0
4,SUPE01017660,SUPA01017664,2008-08-29,DOMEFLAT,DOMEFLAT,W-C-IC,15.0
5,SUPE01017660,SUPA01017665,2008-08-29,DOMEFLAT,DOMEFLAT,W-C-IC,15.0
6,SUPE01017660,SUPA01017666,2008-08-29,DOMEFLAT,DOMEFLAT,W-C-IC,15.0
7,SUPE01017660,SUPA01017667,2008-08-29,DOMEFLAT,DOMEFLAT,W-C-IC,15.0
8,SUPE01017660,SUPA01017668,2008-08-29,DOMEFLAT,DOMEFLAT,W-C-IC,15.0
9,SUPE01017660,SUPA01017669,2008-08-29,DOMEFLAT,DOMEFLAT,W-C-IC,15.0


In [4]:
# Classification & counting
df.groupby(by=['DATA-TYP', 'FILTER01', 'DATE-OBS'], as_index=False).count()

,DATA-TYP,FILTER01,DATE-OBS,EXP-ID,FRAMEID,OBJECT,EXPTIME
0,DOMEFLAT,N-A-L656,2013-11-30,60,60,60,60
1,DOMEFLAT,N-A-L656,2013-12-03,140,140,140,140
2,DOMEFLAT,W-C-IC,2008-08-29,70,70,70,70
3,DOMEFLAT,W-C-IC,2008-08-30,70,70,70,70
4,DOMEFLAT,W-C-IC,2008-08-31,70,70,70,70
5,DOMEFLAT,W-C-IC,2013-12-02,50,50,50,50
6,DOMEFLAT,W-C-IC,2014-04-29,50,50,50,50
7,DOMEFLAT,W-C-RC,2009-08-22,10,10,10,10
8,DOMEFLAT,W-C-RC,2009-08-23,50,50,50,50
9,DOMEFLAT,W-C-RC,2013-12-01,60,60,60,60


### Arranging the Raw Images

In [5]:
obj = (df["DATA-TYP"] == "OBJECT")
flat = (df["DATA-TYP"] == "DOMEFLAT")

In [6]:
df["FILTER01"].unique()

array(['W-C-IC', 'W-J-B', 'W-C-RC', 'N-A-L656'], dtype=object)

In [7]:
# For the flat, object paths
for root_path in [dir_Flat, dir_Obj]:
    for flt in df["FILTER01"].unique():
        obsdate = df["DATE-OBS"][obj & (df["FILTER01"] == flt)].unique()
        if (glob.glob(root_path+flt) == []):
            os.system("mkdir "+root_path+flt)
        else:
            os.system("rm -rfv "+root_path+flt)
            os.system("mkdir "+root_path+flt)
        for date in obsdate:
            os.system("mkdir "+root_path+flt+"/"+date)
            os.system("mkdir "+root_path+flt+"/"+date+"/Images")

In [8]:
# Checking the exposure time of object images
for flt in df["FILTER01"].unique():
    expt_series = df["EXPTIME"][obj & (df["FILTER01"] == flt)]
    expt_uniq = expt_series.unique()
    N_exp = expt_series.value_counts(sort=False).values
    tot_exp = 0
    message = flt+": "
    for NN in np.arange(len(N_exp)):
        tot_exp += expt_uniq[NN]*N_exp[NN]/10
        if (NN < np.arange(len(N_exp))[-1]):
            message += f"{expt_uniq[NN]:.1f} sec x {N_exp[NN]//10:d} + "
        else:
            message += f"{expt_uniq[NN]:.1f} sec x {N_exp[NN]//10:d} = {tot_exp:.1f} sec"
    print(message)

W-C-IC: 180.0 sec x 8 + 60.0 sec x 12 + 240.0 sec x 4 + 120.0 sec x 7 = 3960.0 sec
W-J-B: 240.0 sec x 7 + 200.0 sec x 1 + 60.0 sec x 8 + 180.0 sec x 9 = 3980.0 sec
W-C-RC: 560.0 sec x 1 + 60.0 sec x 4 + 240.0 sec x 3 + 120.0 sec x 9 = 2600.0 sec
N-A-L656: 120.0 sec x 5 + 480.0 sec x 9 = 4920.0 sec


In [9]:
# Moving all the object images
for flt in df["FILTER01"].unique():
    print("\n--- "+flt+" band ---")
    for date in df["DATE-OBS"][obj & (df["FILTER01"] == flt)].unique():
        expid = df["EXP-ID"][obj & (df["FILTER01"] == flt) & (df["DATE-OBS"] == date)]
        imgid = expid.str[:-1].str.replace('SUPE', 'SUPA').unique()
        for i in imgid:
            os.system("cp -rpv "+dir_Raw+i+"*.fits "+dir_Obj+flt+"/"+date+"/Images/")
        print(date, np.sum(obj & (df["FILTER01"] == flt) & (df["DATE-OBS"] == date))//10)


--- W-C-IC band ---
2008-08-30 4
2013-12-04 12
2014-04-29 15

--- W-J-B band ---
2008-09-03 3
2009-08-21 1
2013-12-04 8
2014-04-29 13

--- W-C-RC band ---
2009-08-21 1
2013-12-03 16

--- N-A-L656 band ---
2013-12-02 14


In [10]:
# Matching observation dates & moving matched flat images
for flt in df["FILTER01"].unique():
    print("\n--- "+flt+" band ---")
    for date_obj in df["DATE-OBS"][obj & (df["FILTER01"] == flt)].unique():
        date_obj_df = pd.to_datetime(date_obj)
        flat_obj_df = pd.to_datetime(df["DATE-OBS"][flat & (df["FILTER01"] == flt)].unique())
        idx_matched = np.abs(date_obj_df - flat_obj_df).argmin()
        flat_date = df["DATE-OBS"][flat & (df["FILTER01"] == flt)].unique()[idx_matched]
        
        expid = df["EXP-ID"][flat & (df["FILTER01"] == flt) & (df["DATE-OBS"] == flat_date)]
        imgid = expid.str[:-1].str.replace('SUPE', 'SUPA').unique()
        for i in imgid:
            os.system("cp -rpv "+dir_Raw+i+"*.fits "+dir_Flat+flt+"/"+date_obj+"/Images/")
        print(flat_date, np.sum(flat & (df["FILTER01"] == flt) & (df["DATE-OBS"] == flat_date))//10)


--- W-C-IC band ---
2008-08-30 7
2013-12-02 5
2014-04-29 5

--- W-J-B band ---
2008-08-31 7
2009-08-23 6
2013-12-02 5
2014-04-29 5

--- W-C-RC band ---
2009-08-22 1
2013-12-03 5

--- N-A-L656 band ---
2013-12-03 14


### Writing the SDFRED2 scripts

In [11]:
# Printing the paths of Flat and Object
path_list_flat = sorted(glob.glob(dir_Flat+"*/*"))
path_list_obj = sorted(glob.glob(dir_Obj+"*/*"))

In [12]:
path_list_flat

['/data/jlee/DATA/Subaru/NGC6946/Flat/N-A-L656/2013-12-02',
 '/data/jlee/DATA/Subaru/NGC6946/Flat/W-C-IC/2008-08-30',
 '/data/jlee/DATA/Subaru/NGC6946/Flat/W-C-IC/2013-12-04',
 '/data/jlee/DATA/Subaru/NGC6946/Flat/W-C-IC/2014-04-29',
 '/data/jlee/DATA/Subaru/NGC6946/Flat/W-C-RC/2009-08-21',
 '/data/jlee/DATA/Subaru/NGC6946/Flat/W-C-RC/2013-12-03',
 '/data/jlee/DATA/Subaru/NGC6946/Flat/W-J-B/2008-09-03',
 '/data/jlee/DATA/Subaru/NGC6946/Flat/W-J-B/2009-08-21',
 '/data/jlee/DATA/Subaru/NGC6946/Flat/W-J-B/2013-12-04',
 '/data/jlee/DATA/Subaru/NGC6946/Flat/W-J-B/2014-04-29']

In [13]:
path_list_obj

['/data/jlee/DATA/Subaru/NGC6946/Object/N-A-L656/2013-12-02',
 '/data/jlee/DATA/Subaru/NGC6946/Object/W-C-IC/2008-08-30',
 '/data/jlee/DATA/Subaru/NGC6946/Object/W-C-IC/2013-12-04',
 '/data/jlee/DATA/Subaru/NGC6946/Object/W-C-IC/2014-04-29',
 '/data/jlee/DATA/Subaru/NGC6946/Object/W-C-RC/2009-08-21',
 '/data/jlee/DATA/Subaru/NGC6946/Object/W-C-RC/2013-12-03',
 '/data/jlee/DATA/Subaru/NGC6946/Object/W-J-B/2008-09-03',
 '/data/jlee/DATA/Subaru/NGC6946/Object/W-J-B/2009-08-21',
 '/data/jlee/DATA/Subaru/NGC6946/Object/W-J-B/2013-12-04',
 '/data/jlee/DATA/Subaru/NGC6946/Object/W-J-B/2014-04-29']

In [14]:
# Writing the flat script
text_red1_flat = "# Reset\n"
text_red1_flat += "rm -rfv *.fits *.lis tmp* blankmap*\n"
text_red1_flat += "ln -s Images/*.fits .\n\n"
text_red1_flat += "# Step 1 : Initial Data Inspection and Renaming of Data Frames\n"
text_red1_flat += "ls -1 *.fits > namechange.lis\n"
text_red1_flat += "namechange.csh namechange.lis\n\n"
text_red1_flat += "# Step 2 : Subtraction Overscan and Bias\n"
text_red1_flat += "ls -1 H*.fits > overscansub.lis\n"
text_red1_flat += "overscansub.csh overscansub.lis\n\n"
text_red1_flat += "# Step 3 : Making Flat Field Frames\n"
text_red1_flat += "ls -1 To_RH*.fits > mkflat.lis\n"
text_red1_flat += "mask_mkflat_HA.csh mkflat.lis dome 0.4 1.3\n\n"
text_red1_flat += "# Removing intermediate files\n"
text_red1_flat += "rm -rfv *To*.fits tmp*\n\n"

with open("red1_flat.sh", "w") as f:
    f.write(text_red1_flat)

for i, di in enumerate(path_list_flat):
    os.system("cp -rpv red1_flat.sh "+di+"/red1.sh")
    with open(di+"/red1.sh", "a") as f:
        f.write("# Moving to the object directory\n")
        f.write("cd "+path_list_obj[i]+"\n")

In [15]:
# Writing the object script
text1_red1_obj = "# Reset\n"
text1_red1_obj += "rm -rfv *.fits *.lis tmp* blankmap* *.cat *.dat *.tmp *.mes *.messel\n"
text1_red1_obj += "ln -s Images/*.fits .\n\n"
text1_red1_obj += "# Step 1 : Initial Data Inspection and Renaming of Data Frames\n"
text1_red1_obj += "ls -1 *.fits > namechange.lis\n"
text1_red1_obj += "namechange.csh namechange.lis\n\n"
text1_red1_obj += "# Step 2 : Subtraction Overscan and Bias\n"
text1_red1_obj += "ls -1 H*.fits > overscansub.lis\n"
text1_red1_obj += "overscansub.csh overscansub.lis\n\n"
text1_red1_obj += "# Step 4 : Flat Fielding\n"

text2_red1_obj = "ls -1 dome_mflat*.fits > ffield_mf.lis\n"
text2_red1_obj += "ls -1 To_RH*.fits > ffield_im.lis\n"
text2_red1_obj += "ffield.csh ffield_mf.lis ffield_im.lis\n\n"
text2_red1_obj += "# Step 5 : Distortion Correction and Atmospheric Dispersion Correction\n"
text2_red1_obj += "ls -1 fTo_RH*.fits > distcorr.lis\n"
text2_red1_obj += "distcorr.csh distcorr.lis\n\n"
text2_red1_obj += "# Step 6 : Measurement of PSF size\n"
text2_red1_obj += "ls -1 gfTo_RH*.fits > fwhmpsf_batch.lis\n"
text2_red1_obj += "fwhmpsf_batch.csh fwhmpsf_batch.lis 50 2000 40000 2.0 7.0 > fwhmpsf_batch.log\n"

with open("red1_obj.sh", "w") as f:
    f.write(text1_red1_obj)

for i, di in enumerate(path_list_obj):
    os.system("cp -rpv red1_obj.sh "+di+"/red1.sh")
    with open(di+"/red1.sh", "a") as f:
        f.write("ln -s "+path_list_flat[i]+"/dome_mflat*.fits .\n")
        f.write(text2_red1_obj)